<a href="https://colab.research.google.com/github/Madarasanj/minor-2/blob/main/Copy_of_minor_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install np_utils


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for np_utils: filename=np_utils-0.6.0-py3-none-any.whl size=56441 sha256=04725cf2d7e6022dc818403600e78e7873c889976b743e4a4bd2e2a3d2686f8d
  Stored in directory: /root/.cache/pip/wheels/b6/c7/50/2307607f44366dd021209f660045f8d51cb976514d30be7cc7
Successfully built np_utils


In [ ]:
!python -m nltk.downloader stopwords

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
!python -m nltk.downloader wordnet

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import re
import unicodedata
import itertools
from collections import Counter
from sklearn.preprocessing import LabelEncoder
import np_utils
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Conv1D, MaxPooling1D, Flatten
from keras.optimizers import RMSprop, Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Constants and parameters
stop = stopwords.words('english')
sequence_length = 20  # Adjust as per your data
embedding_dim = 100  # Adjust as per your data
dropout_prob = [0.1]  # Adjust as per your data
val_split = 0.1  # Validation split ratio
batch_size = 50  # Batch size for training

# Function to preprocess text data
def clean_str(string):
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", "", string)
    string = re.sub(r"\)", "", string)
    string = re.sub(r"\?", "", string)
    string = re.sub(r"/", "", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()

# Function to remove numbers and tokenize text
def number_removal(row):
    data1 = row['tweet']
    if type(data1) not in [int, float]:
        line = re.sub(r"[^A-Za-z\s]", " ", data1.strip())
        tokens = line.split()
    else:
        tokens = []
    return ' '.join(tokens)

# Function to generate word frequency
def generate_word_frequency(row):
    data1 = row['tweet']
    tokens = nltk.wordpunct_tokenize(data1)
    token_list = []
    for token in tokens:
        token_list.append(token.lower())
    return ','.join(token_list)

# Function to lemmatize words
def lemmatize_text(tokens):
    wordnet_lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [wordnet_lemmatizer.lemmatize(token) for token in tokens.split(',')]
    return ' '.join(lemmatized_tokens)

# Preprocessing and cleaning data
def preprocess_data(data):
    data['tweet'] = data.apply(number_removal, axis=1)
    data['tokens'] = data.apply(generate_word_frequency, axis=1)
    data['tweet_lem'] = data['tokens'].apply(lemmatize_text)
    return data['tweet_lem']

# Pad sentences to the same length
def pad_sentences(sentences, sequence_length, padding_word="<PAD/>"):
    padded_sentences = []
    for sentence in sentences:
        if len(sentence) >= sequence_length:
            new_sentence = sentence[:sequence_length]
        else:
            num_padding = sequence_length - len(sentence)
            new_sentence = sentence + [padding_word] * num_padding
        padded_sentences.append(new_sentence)
    return padded_sentences

# Build vocabulary
def build_vocab(sentences):
    word_counts = Counter(itertools.chain(*sentences))
    vocabulary_inv = [x[0] for x in word_counts.most_common()]
    vocabulary = {x: i for i, x in enumerate(vocabulary_inv)}
    return vocabulary, vocabulary_inv

# Convert sentences and labels to vectors based on vocabulary
def build_input_data(sentences, labels, vocabulary):
    x = np.array([[vocabulary[str(word)] for word in sentence] if isinstance(sentence, list) else vocabulary[str(sentence)] for sentence in sentences])
    y = np.array(labels)
    return x, y

# Function to load and preprocess data
def load_data(sequence_length):
    data = pd.read_csv("/content/bullying_dataset.csv")
    x_text = preprocess_data(data)
    y = data['label'].to_numpy()
    x_text = [clean_str(sent) for sent in x_text]
    x_text = [s.split(" ") for s in x_text]
    sentences_padded = pad_sentences(x_text, sequence_length)
    vocabulary, _ = build_vocab(sentences_padded)
    x, y = build_input_data(sentences_padded, y, vocabulary)
    return x, y, vocabulary

# Load and preprocess the data
sequence_length = 20  # Adjust as per your data
x, y, vocabulary = load_data(sequence_length)

# Split the data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Define and compile the model (you can choose either LSTM or CNN)
model = Sequential()
model.add(Embedding(len(vocabulary), embedding_dim, input_length=sequence_length))
model.add(LSTM(200, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=15, validation_split=val_split, batch_size=batch_size)

# Evaluate the model
scores = model.evaluate(x_test, y_test, verbose=1)
print("Test Accuracy: %.2f%%" % (scores[1] * 100))


Epoch 1/15
127/127 [==============================] - 13s 80ms/step - loss: 0.5998 - accuracy: 0.7128 - val_loss: 0.5963 - val_accuracy: 0.7054
Epoch 2/15
127/127 [==============================] - 10s 81ms/step - loss: 0.5776 - accuracy: 0.7150 - val_loss: 0.5842 - val_accuracy: 0.7054
Epoch 3/15
127/127 [==============================] - 9s 72ms/step - loss: 0.5484 - accuracy: 0.7361 - val_loss: 0.5892 - val_accuracy: 0.7153
Epoch 4/15
127/127 [==============================] - 10s 76ms/step - loss: 0.5087 - accuracy: 0.7588 - val_loss: 0.6122 - val_accuracy: 0.7054
Epoch 5/15
127/127 [==============================] - 10s 76ms/step - loss: 0.4609 - accuracy: 0.7934 - val_loss: 0.6299 - val_accuracy: 0.6416
Epoch 6/15
127/127 [==============================] - 8s 67ms/step - loss: 0.4062 - accuracy: 0.8248 - val_loss: 0.7642 - val_accuracy: 0.6431
Epoch 7/15
127/127 [==============================] - 11s 87ms/step - loss: 0.3623 - accuracy: 0.8484 - val_loss: 0.8055 - val_accuracy: 0